In [501]:
from pprint import pprint 

from elasticsearch import Elasticsearch

es = Elasticsearch()

# names
yowl = 'yowl'
restaurant = 'restaurant'

location = 'location'
name = 'name'
cuisine_high_q = 'cuisine_high_q'
cuisine_low_q = 'cuisine_low_q'
menu = 'menu'
description = 'description'
price = 'price'
rating = 'rating'
has_discount = 'has_discount'
promoted = 'promoted'
engaged = 'engaged'

pizza_hit = 'pizza hit'
wednesdays = 'wednesdays'
burger_kink = 'burger kink'
apples_bees = 'apples bees'

def recreate_index():
    try:
        es.indices.delete(yowl)
    except:
        pass

    body = {
        "mappings": {
          restaurant: {
            "properties": {
              location: {
                "type": "geo_point"}}}}}
    es.indices.create(yowl, body)

def index_docs(docs):
    recreate_index()
    for doc in docs:
        es.index(yowl,restaurant,doc)
    es.indices.refresh(yowl)
    
def search(query):
    docs = es.search(yowl, restaurant, query)['hits']['hits']
    docs = [dict(doc['_source'],score=doc['_score']) for doc in docs]
    pprint(docs)
    
def assert_in(query, names):
    """
    asserts all named restaurants are in the results
    """
    if isinstance(names,basestring):
        names = [names]
    result = es.search(yowl,restaurant,body=query)
    docs = result['hits']['hits']
    names_in_result = [ doc['_source']['name'] for doc in docs]
    for name in names:
        assert name in names_in_result
        
def assert_not_in(query, names):
    """
    asserts all named restaurants are in the results
    """
    if isinstance(names,basestring):
        names = [names]
    result = es.search(yowl,restaurant,body=query)
    docs = result['hits']['hits']
    names_in_result = [ doc['_source']['name'] for doc in docs]
    for name in names:
        assert not name in names_in_result
        
def assert_first(query, name):
    """
    asserts all named restaurants are in the results
    """
    result = es.search(yowl,restaurant,body=query)
    docs = result['hits']['hits']
    first_name = docs[0]['_source']['name']
    assert name == first_name


### Content

In [502]:
index_docs([
    {
        name: pizza_hit,
        cuisine_low_q: 'italian pizza',
    },
    {
        name: apples_bees,
        description: 'taco pizza',
    }
])

query = {
    'query': {
        'multi_match': {
            'query': 'pizza',
            'fields': ['name^10', 'cuisine_high_q^10', 'cuisine_low_q^4', 'menu^2', 'description^1'],
            'tie_breaker': 0.3,
        },
    }
}
search(query)
assert_in(query, [pizza_hit, apples_bees])
assert_first(query, pizza_hit)

[{u'cuisine_low_q': u'italian pizza',
  u'name': u'pizza hit',
  'score': 0.065443814},
 {u'description': u'taco pizza',
  u'name': u'apples bees',
  'score': 0.0055905404}]


### Customer Preferences

In [503]:
index_docs([
    {
        name: burger_kink,
        price: 'D',
        rating: 'SSS',
    },
    {
        name: apples_bees,
        price: 'DDD',
        rating: 'SSS',
    },
    {
        name: wednesdays,
        price: 'DDD',
        rating: 'S',
    }
])

query = { 
    'query': {
        'bool': {
            'filter': [
                {'match':{
                    price: 'DDD',
                }},
                {'match':{
                    rating: 'SSS',
                }}
            ]
        }
    }
}
search(query)
assert_in(query, apples_bees)
assert_not_in(query, [burger_kink, wednesdays])

[{u'name': u'apples bees', u'price': u'DDD', u'rating': u'SSS', 'score': 0.0}]


### Location

In [504]:
index_docs([
    {
        name: apples_bees,
        location: {
            'lat': 36.15,
            'lon': -86.78
        }
    },
    {
        name: burger_kink,
        location: {
            'lat': 36.40,
            'lon': -86.78
        }
    },
    {
        name: wednesdays,
        location: {
            'lat': 36.15,
            'lon': -86.90
        }
    },
])

query = {
    'filter': {
        'geo_bounding_box': { 
            location: {
                'top_left': {
                    'lat': 36.35,
                    'lon': -86.88,
                },
                'bottom_right': {
                    'lat': 36.05,
                    'lon': -86.68,
                }
            } 
        }
    }
}
search(query)
assert_in(query, apples_bees)
assert_not_in(query, [burger_kink, wednesdays])

[{u'location': {u'lat': 36.15, u'lon': -86.78},
  u'name': u'apples bees',
  'score': 1.0}]


In [505]:
# boost
query = {
    'query': {
        'function_score': {
            'functions': [{
                'gauss': {
                    location: { 
                        'origin': {
                                'lat': 36.154547,
                                'lon': -86.782277
                        },
                        'offset': '0km',
                        'scale':  '10km'
                    }
                }
            }
        ]}
    }
}
search(query)
assert_in(query, [apples_bees, burger_kink, wednesdays])
assert_first(query, apples_bees)

[{u'location': {u'lat': 36.15, u'lon': -86.78},
  u'name': u'apples bees',
  'score': 0.99794066},
 {u'location': {u'lat': 36.15, u'lon': -86.9},
  u'name': u'wednesdays',
  'score': 0.4602034},
 {u'location': {u'lat': 36.4, u'lon': -86.78},
  u'name': u'burger kink',
  'score': 0.0057236874}]


### Business

In [506]:
index_docs([
    {
        name: apples_bees,
        has_discount: True,
        promoted: True,
        engaged: True,
    },
    {
        name: burger_kink,
        has_discount: True,
    },
    {
        name: wednesdays,
        has_discount: True,
        promoted: True,
    }
])

query = {
    'query': {
        'function_score': {
            'functions': [{
                'filter': {
                    'query_string': {
                        'query': 'has_discount:* promoted:* engaged:*'
                    }
                },
                'script_score' : {
                    'params': {
                       'business_boost': 0.1,
                    },
                    'script': """
                        (
                            0.3*doc['has_discount'].value +
                            0.5*doc['promoted'].value +
                            0.2*doc['engaged'].value
                        )*business_boost + 1
                        """
                }
            }
        ]}
    }
}
search(query)
assert_first(query, apples_bees)

[{u'engaged': True,
  u'has_discount': True,
  u'name': u'apples bees',
  u'promoted': True,
  'score': 1.1},
 {u'has_discount': True,
  u'name': u'wednesdays',
  u'promoted': True,
  'score': 1.08},
 {u'has_discount': True, u'name': u'burger kink', 'score': 1.03}]


### Combined

In [507]:
index_docs([
    {
        name: burger_kink,
        location: {
            'lat': 36.15,
            'lon': -86.78
        },
        price: 'DDD',
        rating: 'SSS',
        name: apples_bees,
        description: 'taco pizza',
        cuisine_low_q: 'italian pizza',
    }
])

query = {
    'filter': {
        'bool': {
            'filter': [
                { # location filter
                    'geo_bounding_box': { 
                        location: {
                            'top_left': {
                                'lat': 36.35,
                                'lon': -86.88,
                            },
                            'bottom_right': {
                                'lat': 36.05,
                                'lon': -86.68,
                            }
                        } 
                    }
                },
                { # customer preference
                    'match':{
                        'price': 'DDD',
                }},
                { # customer preference
                    'match':{
                        'rating': 'SSS',
                }}
            ]}
    },
    'query': {
        'function_score': {
            'query': { # content
                'multi_match': {
                    'query': 'pizza',
                    'fields': ['name^10', 'cuisine_high_q^10', 'cuisine_low_q^4', 'menu^2', 'description^1'],
                },
            },
            'functions': [
                { # business concerns
                    'filter': {
                        'query_string': {
                            'query': 'has_discount:* promoted:* engaged:*'
                        }
                    },
                    'script_score' : {
                        'params': {
                           'business_boost': 0.1,
                        },
                        'script': """
                            (
                                0.3*doc['has_discount'].value +
                                0.5*doc['promoted'].value +
                                0.2*doc['engaged'].value
                            )*business_boost + 1
                            """
                    }
                },
                { # location
                    'gauss': {
                        'location': { 
                            'origin': {
                                    'lat': 36.154547,
                                    'lon': -86.782277
                            },
                            'offset': '0km',
                            'scale':  '10km'
                        }
                    }
                }
            ]
        }
    }
}
search(query)
assert_in(query, apples_bees)

[{u'cuisine_low_q': u'italian pizza',
  u'description': u'taco pizza',
  u'location': {u'lat': 36.15, u'lon': -86.78},
  u'name': u'apples bees',
  u'price': u'DDD',
  u'rating': u'SSS',
  'score': 0.023491185}]
